In [ ]:
#lambda for specific object:


import json
import boto3

# Initialize S3 client
s3_client = boto3.client('s3')

def lambda_handler(event, context):
    # Define your bucket name and object key
    bucket_name = 'visiting-card-compiler'
    object_key = 'WhatsApp Image 2024-09-04 at 4.51.12 AM.jpeg'
    
    # Attempt to read the object
    try:
        response = s3_client.get_object(Bucket=bucket_name, Key=object_key)
        object_content = response['Body'].read().decode('utf-8')  # Assuming the object is a text file
        print(f"Successfully read object: {object_key}")
        print(f"Object content: {object_content}")
    except Exception as e:
        print(f"Error accessing S3 object: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps({
                'message': f"Failed to access object {object_key} in bucket {bucket_name}. Error: {str(e)}"
            })
        }
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': f"Successfully accessed object {object_key} in bucket {bucket_name}",
            'content': object_content
        })
    }


In [ ]:
# for checking the access for s3 through lambda :

import json
import boto3

# Initialize S3 client
s3_client = boto3.client('s3')

def lambda_handler(event, context):
    # Specify your bucket name
    bucket_name = 'visiting-card-compiler'
    
    # Attempt to list the objects in the bucket
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            print(f"Objects in bucket {bucket_name}:")
            for obj in response['Contents']:
                print(obj['Key'])
        else:
            print(f"No objects found in bucket {bucket_name}.")
    except Exception as e:
        print(f"Error accessing S3 bucket: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps({
                'message': f"Failed to access bucket {bucket_name}. Error: {str(e)}"
            })
        }
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': f"Successfully accessed bucket {bucket_name}",
            'objects': response.get('Contents', [])
        })
    }


In [ ]:
#lambda code for using textract and exciting iamge uri:


import json
import boto3
import urllib.parse

# Initialize Textract client
textract_client = boto3.client('textract')

def lambda_handler(event, context):
    # Define your existing S3 bucket name and image key
    bucket_name = 'visiting-card-compiler'
    document_name = 'WhatsApp Image 2024-09-04 at 4.51.12 AM.jpeg'
    
    # Log the file being processed
    print(f"Processing file: s3://{bucket_name}/{document_name}")
    
    # Call Textract to detect text in the image
    try:
        response = textract_client.detect_document_text(
            Document={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': document_name
                }
            }
        )
    except Exception as e:
        print(f"Error calling Textract: {e}")
        raise e
    
    # Extract detected text
    extracted_text = ''
    for block in response['Blocks']:
        if block['BlockType'] == 'LINE':
            extracted_text += block['Text'] + '\n'

    # Log the extracted text
    print(f"Extracted Text: {extracted_text}")
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': 'Text extraction completed successfully',
            'extracted_text': extracted_text
        })
    }


In [ ]:
# for full process :


import json
import boto3
import urllib.parse

# Initialize Textract and S3 client
s3_client = boto3.client('s3')
textract_client = boto3.client('textract')

def lambda_handler(event, context):
    # Get bucket name and image key from the S3 event trigger
    bucket_name = event['Records'][0]['s3']['bucket']['name']
    document_name = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')
    
    print(f"Processing file: s3://{bucket_name}/{document_name}")
    
    # Call Textract to detect text in the uploaded image
    try:
        response = textract_client.detect_document_text(
            Document={
                'S3Object': {
                    'Bucket': bucket_name,
                    'Name': document_name
                }
            }
        )
    except Exception as e:
        print(f"Error calling Textract: {e}")
        raise e
    
    # Extract the detected text
    extracted_text = ''
    for block in response['Blocks']:
        if block['BlockType'] == 'LINE':
            extracted_text += block['Text'] + '\n'

    # Log or return the extracted text
    print(f"Extracted Text: {extracted_text}")
    
    return {
        'statusCode': 200,
        'body': json.dumps({
            'message': 'Text extraction completed successfully',
            'extracted_text': extracted_text
        })
    }
